In [1]:
import pandas as pd
import numpy as np
import sqlite3
import scipy as sp
import sklearn as skl

from sklearn.svm import SVC
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.neural_network import MLPRegressor,MLPClassifier
import sklearn.model_selection as ms
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_validate,KFold
from sklearn import metrics
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
import keras
from keras import optimizers,regularizers


C:\Users\Umair\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
engine = sqlite3.connect('CleanData-Apr15')

totaldata_df = pd.read_sql('select * from data', engine)
train_data, test_data= ms.train_test_split(totaldata_df, test_size=0.3, random_state=42)
scaler = StandardScaler()
scaler.fit(train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])
train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']] =scaler.transform(train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])
test_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']] =scaler.transform(test_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])

X_train=train_data.drop(labels=['WIDTH'],axis='columns')
y_train=train_data[['WIDTH']]
X_test=test_data.drop(labels=['WIDTH'],axis='columns')
y_test=test_data[['WIDTH']]


C:\Users\Umair\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Umair\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Umair\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [4]:
def cross_val_keras(input_data,test_data,input_labels,test_labels,layer,reg,act,verbose=0):
    test_labels=test_labels.ravel()
    input_labels=input_labels.ravel()
    err_matnew=np.empty((1,2))
    kf = KFold(n_splits=7,random_state=9, shuffle=True)
    seed = 9
    np.random.seed(seed)
    count=0
    for train_index, test_index in kf.split(input_data):
        X_cvtrain = input_data.iloc[train_index]
        X_cvtest = input_data.iloc[test_index]

        y_cvtrain = input_labels[train_index]
        y_cvtrain=y_cvtrain.squeeze()
        y_cvtrain=y_cvtrain.ravel()

        y_cvtest = input_labels[test_index]
        y_cvtest=y_cvtest.squeeze()
        y_cvtest=y_cvtest.ravel()
                    
        keras_model=keras_nn_model(X_cvtrain.shape[1],act,layer,reg)
        keras_model.fit(x=X_cvtrain, y=y_cvtrain, batch_size=200, epochs=5, verbose=0, validation_split=0, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
        predictions=keras_model.predict(X_cvtest, batch_size=None, verbose=0, steps=None).ravel()
        
        err_vec=np.column_stack((y_cvtest,predictions))
        err_matnew=np.concatenate((err_matnew,err_vec),axis=0)
        temp1=np.square(predictions-y_cvtest)
        temp2=np.abs(predictions-y_cvtest)
        temp3=np.divide(temp2,y_cvtest)
        count+=1
        if verbose==1:
            print('cv error: '+str(count)+' Fold')
            print(np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]]))

    err_matnew=err_matnew[1:,:]     
    
    temp1=np.square(err_matnew[:,1]-err_matnew[:,0])
    temp2=np.abs(err_matnew[:,1]-err_matnew[:,0])
    temp3=np.divide(temp2,err_matnew[:,0])
    cv_error=np.array([np.sqrt(sum(temp1)/temp1.shape[0]),sum(temp2)/temp1.shape[0],100*sum(temp3)/temp1.shape[0]])
    print('cv_error:')
    print(cv_error)
    return cv_error

In [5]:
cvreg_results=[]

In [6]:
def keras_nn_model(dim,act,l1,reg=1e-5):
    model = Sequential()
    model.add(Dense(l1, input_dim=dim, kernel_initializer='uniform', activation=act,kernel_regularizer=regularizers.l2(reg)))
#    model.add(Dense(l2, kernel_initializer='uniform', activation=act))
#    model.add(Dense(5, kernel_initializer='uniform'a, activation=act))
    
    model.add(Dense(1, kernel_initializer='uniform'))
    adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
    model.compile(loss='mean_absolute_percentage_error', optimizer=adam)
    return model

In [16]:
#cross validation to pick the best Neural Net Model for MAPE

count=0
reg=[0.00001, .0001, .001, .01, .1, 0, 1]
layer_size1=[50, 70, 100]
activations=['sigmoid','tanh' ,'relu',]
for act in activations:
    for l1 in layer_size1:
        for r in reg:               
            layer=l1
            alph=r
            err_mat= cross_val_keras(X_train, X_test, np.array(y_train), np.array(y_test), layer=l1,reg=r,act=act,verbose=0)

            cvreg_results.append({'Classifier': 'Keras', 'parameters':'activation='+act+', layer='+str(l1)+',reg='+str(alph) ,'CV RMSE': err_mat[0],'CV MAE':err_mat[1],'CV MPE':err_mat[2]})
            count+=1
            print('count'+str(count))

cv_error:
[ 4.33289059  2.66740269 30.27284794]
count1
cv_error:
[ 4.33340361  2.66773578 30.27563133]
count2
cv_error:
[ 4.33851014  2.67135512 30.30809958]
count3
cv_error:
[ 4.39112071  2.70899061 30.68601826]
count4
cv_error:
[ 4.53846062  2.8259544  31.90596193]
count5
cv_error:
[ 4.33278949  2.66733235 30.27254734]
count6
cv_error:
[ 4.56003136  2.87974293 33.02008369]
count7
cv_error:
[ 4.28719663  2.63509085 29.98433548]
count8
cv_error:
[ 4.28793591  2.63565758 29.98929497]
count9
cv_error:
[ 4.296482    2.64211908 30.04579174]
count10
cv_error:
[ 4.36251311  2.69026843 30.53876655]
count11
cv_error:
[ 4.49584993  2.80048188 31.82219081]
count12
cv_error:
[ 4.28711797  2.63505535 29.98435735]
count13
cv_error:
[ 4.55140757  2.87482727 33.10480855]
count14
cv_error:
[ 4.25401195  2.61352664 29.79291698]
count15
cv_error:
[ 4.25466054  2.61407351 29.8018966 ]
count16
cv_error:
[ 4.26445259  2.62169301 29.89006854]
count17
cv_error:
[ 4.3445027   2.68073138 30.50265173]
count18
c

In [17]:
result=pd.DataFrame(cvreg_results,columns=["Classifier","parameters","CV MAE","CV MPE","CV RMSE"])
result

,Classifier,parameters,CV MAE,CV MPE,CV RMSE
0,Keras,"activation=sigmoid, layer=50,reg=1e-05",2.667399,30.272209,4.332894
1,Keras,"activation=sigmoid, layer=50,reg=0.0001",2.667635,30.275412,4.333271
2,Keras,"activation=sigmoid, layer=50,reg=1e-05",2.667403,30.272848,4.332891
3,Keras,"activation=sigmoid, layer=50,reg=0.0001",2.667736,30.275631,4.333404
4,Keras,"activation=sigmoid, layer=50,reg=0.001",2.671355,30.308100,4.338510
5,Keras,"activation=sigmoid, layer=50,reg=0.01",2.708991,30.686018,4.391121
6,Keras,"activation=sigmoid, layer=50,reg=0.1",2.825954,31.905962,4.538461
7,Keras,"activation=sigmoid, layer=50,reg=0",2.667332,30.272547,4.332789
8,Keras,"activation=sigmoid, layer=50,reg=1",2.879743,33.020084,4.560031
9,Keras,"activation=sigmoid, layer=70,reg=1e-05",2.635091,29.984335,4.287197


In [19]:
result[result['CV MPE']==result['CV MPE'].min()]

,Classifier,parameters,CV MAE,CV MPE,CV RMSE
63,Keras,"activation=relu, layer=100,reg=0",2.495376,28.730787,4.078726
